In [11]:
%run 2_location_estimator.ipynb 

In [12]:
cam_input = [('bh', 'east', 11), ('hp', 'west', 8), ('mg', 'south', 14), ('bm', 'south', 12)]
tri_pairs = triangulation(cam_input)
tri_pairs

[{'cam1': 'bh',
  'cam2': 'mg',
  'estimate': (-116.72201818108532, 33.403355168476644)},
 {'cam1': 'hp',
  'cam2': 'mg',
  'estimate': (-116.74538526354159, 33.273643412769076)},
 {'cam1': 'bm',
  'cam2': 'mg',
  'estimate': (-116.78396098133469, 33.05950781873452)}]

In [13]:
df_intersections = pd.read_csv("../../data/raw/landmarks_processed.csv")

In [14]:
df = pd.read_csv("../../data/raw/landmarks_processed.csv")
df_intersections = df.loc[df['intersection'] == 1].reset_index(drop=True)
test_lms = set(df_intersections['landmark'])
df_intersections

,landmark,camera_abbrev,direction,lat,long,x_pixel,y_pixel,x_res,y_res,intersection,distance,tile
0,otay resevoir - north tip,om,west,32.628680,-116.926238,2315,1153,3072,2048,1,8.518187,24
1,otay resevoir - north tip,sm,south,32.628680,-116.926238,1410,1250,3072,2048,1,7.641891,22
2,mt laguna water tower,ml,south,32.875975,-116.414988,765,637,2048,1536,1,1.892158,12
3,nucal egg ranch,mg,south,33.072523,-116.776892,1731,1117,3072,2048,1,13.023886,14
4,nucal egg ranch,bm,south,33.072523,-116.776892,1067,1127,3072,2048,1,10.144026,12
5,Palomar Observatory right,hp,west,33.348388,-116.859694,161,693,2048,1536,1,2.720259,9
6,Palomar Observatory right,bh,east,33.348388,-116.859694,1044,1011,3072,2048,1,5.747043,12
7,target - san clemente,sclm,north,33.464032,-117.605505,642,1183,3072,2048,1,3.704763,19
8,target - san clemente,sjh,south,33.464032,-117.605505,1681,1152,3072,2048,1,4.332222,22
9,santa ynez reservoir,dwpgm,south,34.072078,-118.567900,2738,1321,3072,2048,1,2.375986,26


In [38]:
acc_list = []
for lm in test_lms:
    data1 = df_intersections.loc[df_intersections['landmark'] == lm].iloc[0]
    data2 = df_intersections.loc[df_intersections['landmark'] == lm].iloc[1]
    
    cam_input = [(data1['camera_abbrev'], data1['direction'], data1['tile']), (data2['camera_abbrev'], data2['direction'], data2['tile'])]
    
    tri_pairs = triangulation(cam_input)
    dist_acc = haversine((data1['long'], data1['lat']), (tri_pairs[0]['estimate'][0], tri_pairs[0]['estimate'][1]), unit=Unit.MILES)
    acc_list.append(dist_acc)
    
    print("Test Point: ",lm)
    print("X: {:2f}, Y: {:2f}".format(tri_pairs[0]['estimate'][0], tri_pairs[0]['estimate'][1]))
    #print("X: {}, Y: {}".format(x_coord, y_coord))
    print("Estimate Distance From Actual: {:2f}mi".format(dist_acc))
    print("\n")

results_df = pd.DataFrame()
results_df['samples'] = [len(acc_list)]
results_df['mean'] = [np.mean(acc_list)]
results_df['median'] = [np.median(acc_list)]
results_df['max'] = [np.max(acc_list)]
results_df['min'] = [np.min(acc_list)]
results_df['% < 3mi'] = [(len([x for x in acc_list if x < 3])/len(acc_list)) * 100]
print(results_df)

Test Point:  santa monica pier
X: -118.510222, Y: 34.010570
Estimate Distance From Actual: 0.729797mi


Test Point:  santa ynez reservoir
X: -118.548760, Y: 34.086405
Estimate Distance From Actual: 1.404585mi


Test Point:  otay resevoir - north tip
X: -116.934332, Y: 32.630586
Estimate Distance From Actual: 0.562442mi


Test Point:  mt laguna water tower
X: -116.421414, Y: 32.895486
Estimate Distance From Actual: 0.746254mi


Test Point:  nucal egg ranch
X: -116.783961, Y: 33.059508
Estimate Distance From Actual: 0.634603mi


Test Point:  veritiv distribution warehouse
X: -117.114122, Y: 33.109818
Estimate Distance From Actual: 0.112704mi


Test Point:  Palomar Observatory right
X: -116.862564, Y: 33.342776
Estimate Distance From Actual: 0.264587mi


Test Point:  tree cluster end
X: -116.789931, Y: 33.178537
Estimate Distance From Actual: 0.605572mi


Test Point:  target - san clemente
X: -117.601856, Y: 33.450377
Estimate Distance From Actual: 0.504680mi


Test Point:  palomar medica